#  01 - Ingesta y Preprocesamiento
Este notebook carga los datos de pings geolocalizados, realiza limpieza de outliers y genera un conjunto de datos base para análisis.


In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import Window
from pyspark.sql.functions import lag, col, from_unixtime, hour, from_utc_timestamp


#imports para graficos
import pandas as pd
import matplotlib.pyplot as plt

In [0]:
df_raw = spark.table("sv_12_2023")
df_raw.display()

In [0]:
df = df_raw.withColumn("datetime", from_utc_timestamp(from_unixtime(col("timestamp")), "America/El_Salvador"))
df = df.withColumn("hour", hour("datetime"))
df.display()

In [0]:
row_count = df.count()
print(f"Registros totales: {row_count}")

Eliminando duplicados en caso existan

In [0]:
df = df.dropDuplicates(["timestamp", "device_id", "latitude", "longitude"])
row_count = df.count()
print(f"Registros totales (sin duplicados): {row_count}")

Identificando posibles outliers en la data

In [0]:
df_pings_por_dispositivo_dia = df.groupBy("date", "device_id").count() \
    .withColumnRenamed("count", "pings_por_dia")

df_pings_por_dispositivo_dia.display()

In [0]:
# Convertimos a pandas para graficar
pdf1 = df_pings_por_dispositivo_dia.toPandas()

# Histograma
plt.figure(figsize=(10, 5))
plt.hist(pdf1["pings_por_dia"], bins=50, edgecolor='black')
plt.title("Histograma: Pings por dispositivo por día")
plt.xlabel("Número de pings")
plt.ylabel("Frecuencia")
plt.grid(True)
plt.show()

In [0]:
df_dispositivos_por_dia = df.select("date", "device_id").distinct() \
    .groupBy("date").count() \
    .withColumnRenamed("count", "dispositivos_por_dia")

df_dispositivos_por_dia.display()

In [0]:
pdf2 = df_dispositivos_por_dia.toPandas()

plt.figure(figsize=(10, 5))
plt.hist(pdf2["dispositivos_por_dia"], bins=30, edgecolor='black')
plt.title("Histograma: Dispositivos únicos por día")
plt.xlabel("Cantidad de dispositivos")
plt.ylabel("Frecuencia")
plt.grid(True)
plt.show()

Calculando diferencias entre pings por dispositivo, para identificar si hay un muestreo excesivo para limitarlo

In [0]:
# Orden por dispositivo y timestamp
w = Window.partitionBy("device_id").orderBy("timestamp")

# Calcular diferencia con el ping anterior (en segundos)
df_diff = df.withColumn("prev_timestamp", lag("timestamp").over(w)) \
            .withColumn("diff_seconds", col("timestamp") - col("prev_timestamp"))

# Filtrar los valores nulos (primer ping de cada dispositivo no tiene anterior)
df_diff_valid = df_diff.filter(col("diff_seconds").isNotNull())

In [0]:
# Convertimos a pandas para graficar
pdf_diff = df_diff_valid.select("diff_seconds").toPandas()

# Opcional: quitar valores extremos para enfocar el histograma (ej: < 1h = 3600s)
pdf_diff_filtered = pdf_diff[pdf_diff["diff_seconds"] < 3600]

# Histograma
plt.figure(figsize=(10, 5))
plt.hist(pdf_diff_filtered["diff_seconds"], bins=60, edgecolor='black')
plt.title("Histograma: Tiempo entre pings (en segundos)")
plt.xlabel("Segundos desde el ping anterior")
plt.ylabel("Frecuencia")
plt.grid(True)
plt.show()

Probando limitar a un ping por dispositivo por minuto

In [0]:
w = Window.partitionBy("device_id").orderBy("timestamp")

df_diff = df.withColumn("prev_ts", lag("timestamp").over(w)) \
            .withColumn("diff_seconds", col("timestamp") - col("prev_ts")) \
            .filter((col("diff_seconds").isNull()) | (col("diff_seconds") >= 30))  # 60 segundos o más

In [0]:
print(f"Registros antes del filtrado: {df.count()}")
print(f"Registros filtrando pings espaciados < 60s: {df_diff.count()}")

In [0]:
df_diff.write.mode("overwrite").format("delta").saveAsTable("sv_12_2023_limpio_all_pings")

In [0]:
%sql
DESCRIBE DETAIL sv_12_2023_limpio_all_pings

In [0]:
%sql
DESCRIBE HISTORY sv_12_2023_limpio_all_pings